# Text generation with an RNN

## Setup

### Import TensorFlow and other libraries

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

### Read the data

First, look in the text:

In [ ]:
# Read, then decode for py2 compat.
text = open('perfumer.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 395601 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

Парфюмер. История одного убийцы



В городах того времени стояла вонь, почти невообразимая для нас, современных людей. Улицы воняли навозом, дворы воняли мочой, лестницы воняли гнилым деревом и крысиным пометом, кухни — скверным углем и бараньим 


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

117 unique characters


## Process the text

### Vectorize the text

Before training, you need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now you have an integer representation for each character. Notice that you mapped the character as indexes from 0 to `len(unique)`.

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  "'" :   4,
  '(' :   5,
  ')' :   6,
  ',' :   7,
  '-' :   8,
  '.' :   9,
  '0' :  10,
  '1' :  11,
  '2' :  12,
  '3' :  13,
  '4' :  14,
  '5' :  15,
  '6' :  16,
  '7' :  17,
  '8' :  18,
  '9' :  19,
  ...
}


In [ ]:
# Show how the first 8 characters from the text are mapped to integers
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:8]), text_as_int[:8]))

'Парфюмер' ---- characters mapped to int ---- > [ 67  80  96 100 110  92  85  96]


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [ ]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(8):
    print(idx2char[i.numpy()])

П
а
р
ф
ю
м
е
р


The `batch` method lets us easily convert these individual characters to sequences of the desired size.

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(8):
    print(repr(''.join(idx2char[item.numpy()])))

'Парфюмер. История одного убийцы\r\n\r\n\r\n\r\nВ городах того времени стояла вонь, почти невообразимая для на'
'с, современных людей. Улицы воняли навозом, дворы воняли мочой, лестницы воняли гнилым деревом и крыс'
'иным пометом, кухни —\xa0скверным углем и бараньим салом; непроветренные гостиные воняли слежавшейся пыл'
'ью, спальни — грязными простынями, влажными перинами и остро-сладкими испарениями ночных горшков. Из '
'каминов несло верой, из дубилен — едкими щелочами, со скотобоен — выпущенной кровью. Люди воняли пото'
'м и нестираным платьем; изо рта у них пахло сгнившими зубами, из животов — луковым соком, а из тела, '
'когда они старели, начинали пахнуть старым сыром, и кислым молоком, и болезненными опухолями. Воняли '
'реки, воняли площади, воняли церкви, воняло под мостами и во дворцах. Воняли крестьяне и священники, '


For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [ ]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Парфюмер. История одного убийцы\r\n\r\n\r\n\r\nВ городах того времени стояла вонь, почти невообразимая для н'
Target data: 'арфюмер. История одного убийцы\r\n\r\n\r\n\r\nВ городах того времени стояла вонь, почти невообразимая для на'


Each index of these vectors is processed as a one time step. For the input at time step 0, the model receives the index for "F" and tries to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:8], target_example[:8])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 67 ('П')
  expected output: 80 ('а')
Step    1
  input: 80 ('а')
  expected output: 96 ('р')
Step    2
  input: 96 ('р')
  expected output: 100 ('ф')
Step    3
  input: 100 ('ф')
  expected output: 110 ('ю')
Step    4
  input: 110 ('ю')
  expected output: 92 ('м')
Step    5
  input: 92 ('м')
  expected output: 85 ('е')
Step    6
  input: 85 ('е')
  expected output: 96 ('р')
Step    7
  input: 96 ('р')
  expected output: 9 ('.')


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_training.png?raw=1)

Please note that Keras sequential model is used here since all the layers in the model only have single input and produce single output. In case you want to retrieve and reuse the states from stateful RNN layer, you might want to build your model with Keras functional API or model subclassing. Please check [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse) for more details.

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 117) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (64, None, 256)           29952     
_________________________________________________________________
gru_4 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_4 (Dense)              (64, None, 117)           119925    
Total params: 4,088,181
Trainable params: 4,088,181
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [ ]:
sampled_indices

array([110,   9,  74,  31,  63,  93, 104, 101,  82,  53,  81,  24,  63,
       106,  39,  24, 106,  59,   0,  24,  51, 104, 114,  70,  78,  72,
         3,  14,  46, 108,  41,  66, 112,  15, 116,  98,  19,  63,  15,
        43,  16, 107,  61,  67, 100,  77,  60,  62,  88,  75,  26,  98,
       114,  73,  72,  67,  92,  13,  15,  76,  10, 114,  97,  74,  75,
       110,  54,  96,  82,  96, 115, 116,  90,  86,  84,  34,  66,  15,
        56,  20,  66,  96,  75,  40,  22,  76,   8,  58,  95,  10,  85,
       108,  89,  38,   5,  41,  30,  78,  34,  22])

Decode these to see the text predicted by this untrained model:

In [ ]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ыстукивали его, измеряли, заглядывали ему в ром, оттягивали веки. Другие заговаривали с ним, интерес'

Next Char Predictions: 
 'ю.ЦbЛншхвАбEЛъlEъЖ\nE«ш“ТЮФ!4tьnОё5…т9Л5p6ыИПфЭЗКиЧIт“ХФПм35Ш0“сЦЧюБрвр„…кждeО5Г:ОрЧm?Ш-Еп0еьйi(naЮe?'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 117)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.762528


Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [ ]:
EPOCHS = 30

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
61/61 [==============================] - 3s 53ms/step - loss: 3.4111
Epoch 2/30
61/61 [==============================] - 3s 53ms/step - loss: 2.5557
Epoch 3/30
61/61 [==============================] - 3s 53ms/step - loss: 2.3913
Epoch 4/30
61/61 [==============================] - 3s 53ms/step - loss: 2.2699
Epoch 5/30
61/61 [==============================] - 3s 53ms/step - loss: 2.1588
Epoch 6/30
61/61 [==============================] - 3s 54ms/step - loss: 2.0498
Epoch 7/30
61/61 [==============================] - 3s 54ms/step - loss: 1.9436
Epoch 8/30
61/61 [==============================] - 3s 55ms/step - loss: 1.8476
Epoch 9/30
61/61 [==============================] - 3s 55ms/step - loss: 1.7589
Epoch 10/30
61/61 [==============================] - 3s 55ms/step - loss: 1.6818
Epoch 11/30
61/61 [==============================] - 3s 55ms/step - loss: 1.6122
Epoch 12/30
61/61 [==============================] - 3s 55ms/step - loss: 1.5479
Epoch 13/30
61/61 [==================

## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, you need to rebuild the model and restore the weights from the checkpoint.


In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (1, None, 256)            29952     
_________________________________________________________________
gru_5 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_5 (Dense)              (1, None, 117)            119925    
Total params: 4,088,181
Trainable params: 4,088,181
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

The following code block generates the text:

* Begin by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead of only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.


![To generate text the model's output is fed back to the input](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_sampling.png?raw=1)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"Это произошло в один из самых жарких дней года."))

Это произошло в один из самых жарких дней года. Теперь он почаливался.
— В нем Гренуй, он отреал из города, утом не смой пробие, он одется воздойно. И пока он конечно, не было никакой курток и диф, как только он оказался в голубой куртке поднялось его сердце. И он присел на корточки, закрыл глаза. Глаза были некоторых убитых девушек. Зрелище это хотя его невозможно! Вером5о все свичет их, он открыл закупорвоченным и крепкими из шрамыми и мензуркой; пуполько возможного, и желщины слились там, то есть задумалову двере глаза, и, кого оно отромно изучить на улицу под вокруг плоских салан новым. Вездя ты были если бы слоим и судорожной столо, и удли людей не застовал. Он бы я божестновение не давт в своем деле. Тремя хотовое масло, главное кломкое, но мне крыл возмещение около сегодня нетропасть его с ток партно-передал дудой — он достиг некоторой учуял, что покорены померить ты должно беспомощностью, то есть утром он показался, даже сейчас этот и в то же воспоминание о fluidum letale, кото